# For discussion with Prof Gull
- Need to make this faster, ideas:
    - Pool/multiprocessing
- I can replicate all the plots, but the scale is wrong...
- Next steps:
    - Any ideas why my scale might be off? Consider reaching out to authors?
    - Comparing this with ER, networkx, showing how Newman-Ziff works?
    
Boost library for graphs: http://www.boost.org/doc/libs/1_65_1/libs/graph/doc/index.html

# Physics 514, Fall 2017: Studying explosive percolation on directed networks

Outline for report:
- Background
    - Phys 514 concepts: Percolation, critical point, critical exponents
    - Why is explosive percolation different?
    - Network theory background:
        - Networks: Directed v undirected, ordered v unordered, classical graphs (Erdos-Renyi)
        - Clustering: For directed networks, study the strongest connected component, SCC (define)
        - Clustering algorithms: Hoshen-Kopelman won't work here; Newman-Ziff for undirected networks, Tarjan's Algorithm (implemented here) for finding the SCC of directed networks

In [1]:
# Import needed packages
import time
from tqdm import tqdm,tqdm_notebook
import itertools
import numpy as np
import pandas as pd
import scipy
import random
import networkx as nx
from collections import defaultdict
import matplotlib.pyplot as plt
%matplotlib inline

# Helper functions for graphs

*Consider adding these into the class later.*

In [2]:
def check_if_edge_exists(proposed,edges):
    for item in edges:
        if proposed==item: return True
        else: pass

def edges_to_viz(edges,n):
        viz_array = np.zeros((n,n))
        for i in edges:
            viz_array[i[0],i[1]]+=1
        return viz_array

def edges_to_children(edges):
    children = defaultdict(list)
    for u,v in edges:
        children[u].append(v)
    return children

# Initialize graphs

In [3]:
class Graph(object):
  
    def __init__(self,n,m,process_name):
        self.n = n
        self.m = m
        self.edge_density = m/n
        self.edges = []
        self.adjacency_matrix = np.zeros((n,n))
        self.nodes = 0
        self.LJ = 0       
        self.process = process_name 
        
    def initialize(self):
        if self.process=='CODER': self.CODER()
        elif self.process=="ODER": self.ODER()
        else: print('initialize with a valid process')
    
    def add_edge(self,p):
        if self.adjacency_matrix[p[0],p[1]]:
            if self.adjacency_matrix[p[1],p[0]]: pass
            else: self.adjacency_matrix[p[1],p[0]] = 1; self.nodes+=1
        else: self.adjacency_matrix[p[0],p[1]]=1; self.nodes+=1
    
    def add_edge_OLD(self,proposed_edge):
        if check_if_edge_exists(proposed_edge,self.edges):
            if check_if_edge_exists(tuple(reversed(proposed_edge)),self.edges): pass
            else: self.edges.append(tuple(reversed(proposed_edge)))
        else: self.edges.append(proposed_edge)

    def ODER(self):
        # set up list of nodes with ranked order
        n_list = np.linspace(0,self.n-1,self.n).astype(int)
        # add m edges to the plot
#         while len(self.edges)<self.m:
        pbar = tqdm_notebook(total=(self.m-self.nodes),desc="Implementing ODER process")
        while self.nodes<self.m:
            first_node, second_node = random.sample(list(n_list),2)
            proposed_edge = tuple(sorted((first_node,second_node)))
            n0 = self.nodes
            self.add_edge(proposed_edge)
            pbar.update(self.nodes-n0)
        pbar.close()
            
    def CODER(self):
        # set up list of nodes with ranked order
        n_list = np.linspace(0,self.n-1,self.n).astype(int)
        while len(self.edges)<self.m:
            first_node, second_node, third_node = random.sample(list(n_list),3)
            proposed_edges = tuple(itertools.combinations(tuple(sorted((first_node,second_node,third_node))), 2))

            # for nodes with min difference, check if they/their reverse already exist; if no, add them
            difference = np.asarray([nodes[1]-nodes[0] for nodes in proposed_edges])
            idx = np.where(difference == difference.min())[0]
            for i in idx:
                proposed_edge = proposed_edges[i]
                self.add_edge(proposed_edge)

# Finding the strongest connected component using Tarjan's Algorithm

[notes to be cleaned up]

Paper reference: [https://doi.org/10.1137/0201010](https://doi.org/10.1137/0201010)

![here](https://upload.wikimedia.org/wikipedia/commons/6/60/Tarjan%27s_Algorithm_Animation.gif)

From Wikipedia:
- A depth-first search begins from an arbitrary start node
- subsequent depth-first searches are conducted on any nodes that have not yet been found
- As usual with depth-first search, the search visits every node of the graph exactly once, declining to revisit any node that has already been visited
- => the collection of search trees is a spanning forest of the graph

While going through the nodes:
- Nodes are placed on a stack in the order in which they are visited
- A node remains on the stack after it has been visited if and only if there exists a path in the input graph from it to some node earlier on the stack
- At the end of the call that visits v and its descendants, we know whether v itself has a path to any node earlier on the stack. 
    - If yes, the call returns, leaving v on the stack to preserve the invariant
    - If no, then v must be the root of its strongly connected component, which consists of v together with any nodes later on the stack than v (such nodes all have paths back to v but not to any earlier node, because if they had paths to earlier nodes then v would also have paths to earlier nodes which is false)

Book-keeping:
- Each node v is assigned:
    - a unique integer v.index, which numbers the nodes consecutively in the order in which they are discovered
    - a value v.lowlink that represents (roughly speaking) the smallest index of any node known to be reachable from v, including v itself
- Conditions:
    - if v.lowlink < v.index, leave v on stack
    - if v.lowlink == v.index, v must be removed as the root of a strongly connected component
- Calculating: The value v.lowlink is computed during the depth-first search from v, as this finds the nodes that are reachable from v.


In [4]:
# need to rewrite this to be more user-friendly
def tarjan_OLD(neighbors):  
        index_counter = [0]
        index = {}
        lowlink = {}
        stack = []

        result = []

        # v is node-- wait, but shouldn't it actually be u...?
        def calc_component(node):
            # v.index, v.lowlink
            index[node] = index_counter[0]
            lowlink[node] = index_counter[0]
            index_counter[0] += 1
            stack.append(node)

            # find the children
            try:
                children = neighbors[node]
            except:
                children = []
            for child in children:
                # if the child hasn't been visited, run this on it
                print(lowlink)
                if child not in lowlink:
                    calc_component(child)
                    lowlink[node] = min(lowlink[node],lowlink[child])
                # if the child is in the stack, that means they're also in the SCC
                elif child in stack:
                    lowlink[node] = min(lowlink[node],index[child])

            if lowlink[node]==index[node]:
                connected_component = []
                while True:
                    successor = stack.pop()
                    connected_component.append(successor)
                    if successor == node: break
                component = tuple(connected_component)
                # storing the result
                result.append(component)

        for node in list(neighbors):
            if node not in lowlink:
                calc_component(node)

        return result

In [9]:
# need to rewrite this to be more user-friendly
def tarjan(adjacency_matrix):  
        index_counter = [0]
        index = {}
        lowlink = {}
        stack = []

        result = []

        # v is node-- wait, but shouldn't it actually be u...?
        def calc_component(node):
            # v.index, v.lowlink
            index[node] = index_counter[0]
            lowlink[node] = index_counter[0]
            index_counter[0] += 1
            stack.append(node)

            # find the children
            try:
                children = np.where(adjacency_matrix[node,:]==1)[0]
#                 print(children)
            except:
                children = []
            for child in children:
                # if the child hasn't been visited, run this on it
                if child not in lowlink:
                    calc_component(child)
                    lowlink[node] = min(lowlink[node],lowlink[child])
                # if the child is in the stack, that means they're also in the SCC
                elif child in stack:
                    lowlink[node] = min(lowlink[node],index[child])

            if lowlink[node]==index[node]:
                connected_component = []
                while True:
                    successor = stack.pop()
                    connected_component.append(successor)
                    if successor == node: break
                component = tuple(connected_component)
                # storing the result
                result.append(component)

        for node in range(adjacency_matrix.shape[0]):
            if node not in lowlink:
                calc_component(node)

        return result

# Implement search function for jumps from paper

This paper is particularly intreested in finding the explosive percolation phase transition.

While in our classic lattice percolation models, this was a matter of finding $p_c$, here it's going to be an edge density, $\delta=\frac{m}{n}$ where $m$ is the number of edges and $n$ is the number of nodes.

Instead of calculating the SCC every time an edge is added, it builds the completed graph then performs a binary search to find the edge addition at which the largest jump-- the percolation transition-- occured.

In [6]:
def ranked_SCC(connected_components,rank=1):
    C = []
    for members in connected_components:
        C.append(len(members))
    if rank>len(C): return 0
    return sorted(C)[-rank]

def binary_search(graph,start,end,LJ):
    midpoint = int(round((start+end)/2))
    head = ranked_SCC(tarjan(edges_to_children(graph.edges[0:start+1])))
    mid = ranked_SCC(tarjan(edges_to_children(graph.edges[0:midpoint])))
    tail = ranked_SCC(tarjan(edges_to_children(graph.edges[0:end])))
    if abs(end-start)==1:
        if (tail-head)>LJ: LJ = (tail-head)
        return LJ
    elif (mid-head)>(tail-mid):
        if (mid-head)>(graph.n/100):
            LJ = (mid-head)
            return binary_search(graph,start,midpoint,LJ)
    elif (tail-mid)>(graph.n/100):
        LJ = (tail-mid)
        return binary_search(graph,midpoint,end,LJ)
    return LJ

def get_largest_jump(graph):
    start = 0
    midpoint = int(round(graph.m/2))
    end = graph.m-1
#     print(start,midpoint,end)
    head = ranked_SCC(tarjan(edges_to_children(graph.edges[start:start+1])))
    mid = ranked_SCC(tarjan(edges_to_children(graph.edges[start:midpoint])))
    tail = ranked_SCC(tarjan(edges_to_children(graph.edges[start:end])))
    LJ = 0
    if (tail-head)<(graph.n/100): return jump
    elif (mid-head)>(graph.n/100):
        jump = binary_search(graph,start,midpoint,LJ)
    elif (tail-mid)>(graph.n/100):
        jump = binary_search(graph,midpoint,end,LJ)
    return jump

# Implementing the percolation processes 

The paper does an edge density of up to 50, with $10^6$ nodes.

The way the paper defines edge density is
$$\delta = \frac{m}{n}$$
where $m$ is the number of directed edges on the graph, and $n$ is the number of nodes. An edge density of 50 implies that each site has 50 neighbors, which is really dense. 

In [7]:
# n = 10^4 and edge_density = 50 takes 4 min to run
n = 10**4
edge_density = 5e-5*n
m = edge_density*n
# CODER_test = Graph(n,m,'CODER')
ODER_test = Graph(n,m,'ODER')

In [8]:
print(ODER_test.nodes)
print(ODER_test.nodes/ODER_test.m)
print(ODER_test.m)

0
0.0
5000.0


- Running 10^4 nodes takes ~3 seconds (m=5k)
- Running 5x10^4 nodes takes ~1 min (m=125k)
- Running 10^5 nodes should take ~30 minutes (woof)

In [9]:
# CODER_test.initialize()
ODER_test.initialize()

In [10]:
n = 10**2
p = 50/n
er = nx.erdos_renyi_graph(n,p,directed=True)

Visualizing networks of 100 nodes using a graph, especially at an edge density of 50, is likely to be a mess-- and not easy to do. The paper does not attempt to show graphs for their findings, beyond illustrative graphs of their processeds.

Instead, I steal the idea of how co-variance matrices are sometimes shown, and convert the edges of the graph to a binary yes/no edges array. This will show biases in my networks-- e.g. if there are no or fewer reverse edges, the lower traingle of the edge array will be sparser. This is indeed what we see for ODER and C-ODER, which have a preferential ordering rule. 

In [ ]:
# er_viz = edges_to_viz(er.edges(),n)

# plt.imshow(er_viz,cmap='binary')
# plt.title('Erdos-Renyi stochastic graph for comparison')
# plt.ylabel('tail')
# plt.xlabel('head')
# plt.colorbar()
# plt.show()


# ODER_viz = edges_to_viz(ODER_test.edges,ODER_test.n)

plt.imshow(ODER_test.adjacency_matrix,cmap='binary')
plt.title('ODER simulation')
plt.ylabel('tail')
plt.xlabel('head')
plt.colorbar()
plt.show()


# CODER_viz = edges_to_viz(CODER_test.edges,CODER_test.n)

# plt.imshow(CODER_viz,cmap='binary')
# plt.title('CODER simulation')
# plt.ylabel('tail')
# plt.xlabel('head')
# plt.colorbar()
# plt.show()

# Replicating figures from the paper

I wonder if I also need to change the edge density, since I'm not using the same number of nodes.

Maybe the better metric is, what % of all other nodes is each node connected to, on average?

If $\delta=50$ for $n=10^6$, then each node is on average connected to $\frac{50}{10^6}=5e-5$ of the rest of the system.

So for a system of size $n$, we can set an equivalent density with $\frac{\delta}{n}=5e-5$.

Then the total number of edges in the system are $m={\delta}n$

In [11]:
n = np.linspace(1000,10**6,10)
equiv_densities = n*(5*10**(-5))
m = equiv_densities*n
print(n)
print(equiv_densities)
print(m)

[    1000.   112000.   223000.   334000.   445000.   556000.   667000.
   778000.   889000.  1000000.]
[  0.05   5.6   11.15  16.7   22.25  27.8   33.35  38.9   44.45  50.  ]
[  5.00000000e+01   6.27200000e+05   2.48645000e+06   5.57780000e+06
   9.90125000e+06   1.54568000e+07   2.22444500e+07   3.02642000e+07
   3.95160500e+07   5.00000000e+07]


## Fig 5&6

I'm able to replicate the shape of these results, my "edge densities" are off by a factor of $\approx 5$. It's not clear to me why this is.

The equivalent density at these system sizes is much lower. If anything, I'm actually getting a much higher critical density than I should!

In [12]:
def matrix_to_dict(graph):
    children = defaultdict(list)
    for u in range(ODER_test.adjacency_matrix.shape[0]):
        child = np.where(ODER_test.adjacency_matrix[u]==1)[0].tolist()
        children[u].append(child)
    return children

In [7]:
def figure56(process,edge_densities,n):
    largest_scc = []
    Sample = Graph(n,0,process)
    for density in edge_densities:
        Sample.m = density*n
        Sample.initialize()
#         largest_scc.append(ranked_SCC(tarjan(edges_to_children(Sample.edges)))/n)
#         largest_scc.append(ranked_SCC(tarjan(matrix_to_dict(Sample.adjacency_matrix)))/n)
        largest_scc.append(ranked_SCC(tarjan(Sample.adjacency_matrix))/n)
    return largest_scc

In [ ]:
# Plot ODER - might have to run outside of a jupyter notebook to get n-10^5 or higher

replicates = 1
n = 10**5
edge_density = 5e-5*n

# edge_densities = np.linspace(0.1*edge_density,edge_density,10)
# edge_densities = np.linspace(1,50,20)
edge_densities = np.linspace(10**(-6)*n,5*10**(-5)*n,10)
m = edge_densities*n
print(m)

for i in range(replicates):
    plt.plot(edge_densities,figure56('ODER',edge_densities,n))
plt.title('Explosive percolation of ODER process')
plt.ylabel('Largest SCC')
plt.xlabel('Edge density')
plt.show()

[  10000.           64444.44444444  118888.88888889  173333.33333333
  227777.77777778  282222.22222222  336666.66666667  391111.11111111
  445555.55555556  500000.        ]


![fig5](figs/fig5.png)

In [ ]:
# Plot C-ODER

replicates = 10
edge_densities = np.linspace(1,10,20)
for i in range(replicates):
    plt.plot(edge_densities,figure56('CODER',edge_densities))
plt.title('Explosive percolation of C-ODER process')
plt.ylabel('Largest SCC')
plt.xlabel('Edge density')
plt.show()

![fig6](figs/fig6.png)

## Figure 7

Again, in the paper they simulate up to $10^6$ nodes, which is just wildly large.

![fig7](figs/fig7.png)

In [ ]:
def figure7(process,n_sizes):
    edge_density = 10
    max_jump = []
    Sample = Graph(1,1,process)
    for n in n_sizes:
        m = edge_density*n
        Sample.n = n
        Sample.m = m
        Sample.initialize()
        max_jump.append(get_largest_jump(Sample)/n)
    return max_jump

In [ ]:
# Plotting C-ODER coefficient

systems = np.linspace(500,1000,5).astype(int)
replicates = 15
jumps = []
for i in range(replicates):
    jumps.append(figure7('CODER',systems))

CODER_jumps_array = np.asarray(jumps).reshape((replicates,len(jumps[0])))
CODER_vg_jump= np.mean(CODER_jumps_array, axis=0)
CODER_std_jump= np.std(CODER_jumps_array, axis=0)

In [ ]:
plt.errorbar(systems,CODER_vg_jump,yerr=CODER_std_jump, fmt='o')
# plt.axis([0,max(systems)*1.1,-0.05,0.20])
plt.title('Max jump size, C-ODER')
plt.ylabel('Max jump')
plt.xlabel('System size')
plt.show()

In [ ]:
# Plotting ODER coefficient

systems = np.linspace(500,1000,5).astype(int)
replicates = 15
jumps = []
for i in range(replicates):
    jumps.append(figure7('ODER',systems))

ODER_jumps_array = np.asarray(jumps).reshape((replicates,len(jumps[0])))
ODER_avg_jump= np.mean(ODER_jumps_array, axis=0)
ODER_std_jump= np.std(ODER_jumps_array, axis=0)

In [ ]:
plt.errorbar(systems,ODER_avg_jump,yerr=ODER_std_jump, fmt='o')
# plt.axis([0,max(systems)*1.1,-0.05,0.20])
plt.title('Max jump size, ODER')
plt.ylabel('Max jump')
plt.xlabel('System size')
plt.show()

## Figure 8

Again, this spot-on replicates the finding that the C-ODER process jump comes from the combination of two large components combining-- but the critical edge density is off by a factor of 5! 

![fig8](figs/fig8.png)

In [ ]:
def figure8(process,edge_densities):
    n = 10**2
    first_scc,second_scc,third_scc = [],[],[]
    Sample = Graph(n,1,'CODER')
    for density in edge_densities:
        m = density*n
        Sample.m = m
        Sample.initialize()
        first_scc.append(ranked_SCC(tarjan(edges_to_children(Sample.edges)),rank=1)/n)
        second_scc.append(ranked_SCC(tarjan(edges_to_children(Sample.edges)),rank=2)/n)
        third_scc.append(ranked_SCC(tarjan(edges_to_children(Sample.edges)),rank=3)/n)
    return first_scc, second_scc, third_scc

In [ ]:
edge_densities = np.linspace(1,10,20)
first_scc, second_scc, third_scc = figure8('CODER',edge_densities)

plt.scatter(edge_densities,first_scc,label="Largest SCC")
plt.scatter(edge_densities,second_scc,label="Second largest SCC")
plt.scatter(edge_densities,third_scc,label="Third largest SCC")
plt.title('SCC combination near critical edge density in a C-ODER graph')
plt.ylabel('Component size')
plt.xlabel('Edge density')
plt.legend()
plt.show()

## Figure 9
Interesting, but low priority until other issues are worked out.

![fig9](figs/fig9.png)